In [ ]:
# https://github.com/jonathansick/starplex/tree/master/starplex/database/meta

In [ ]:
import tempfile
from astropy.table import Table
import numpy as np
import psycopg2

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, types

In [ ]:
allVisit_path = '/Users/adrian/projects/thejoker/data/allVisit-l30e.2.fits'
tbl = Table.read(allVisit_path, format='fits', hdu=1)

In [ ]:
engine = create_engine('postgresql://adrian@localhost:5432/apogee')

In [ ]:
numpy_type_map = dict()
numpy_type_map[np.float32] = types.Numeric
numpy_type_map[np.float64] = types.Numeric
numpy_type_map[np.int16] = types.SmallInteger
numpy_type_map[np.int32] = types.Integer
numpy_type_map[np.int64] = types.BigInteger
numpy_type_map[np.str_] = types.String

In [ ]:
Base = declarative_base()

class AllVisit(Base):
    __tablename__ = 'allvisit'

    id = Column(Integer, primary_key=True)
    
    def __repr__(self):
        return "<ApogeeVisit(APOGEE_ID='{}', MJD='{}')>".format(self.apogee_id, self.mjd)
    
for name in tbl.columns:
    dtype = numpy_type_map[tbl[name].dtype.type]
    c = Column(name.lower(), dtype)
    setattr(AllVisit, name.lower(), c)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
with engine.begin() as connection:
    connection.execute(AllVisit.

In [ ]:
tbl.dtype

In [ ]:
tbl.colnames

In [ ]:
tbl.colnames

In [ ]:
with psycopg2.connect(database='apogee', user='adrian', host='localhost') as conn:
    with conn.cursor() as cursor:        
        with tempfile.TemporaryFile(mode='r+') as f:
            tbl[:8].write(f, format='ascii.fast_no_header', delimiter=',')
            f.seek(0)

            cursor.copy_from(f, 'allvisit', sep=",", columns=tbl.colnames)